<a href="https://colab.research.google.com/github/crimama/DL_study/blob/main/unet_segmentation_practice_road.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 데이터 다운로드

In [ ]:
!pip install tensorflow-datasets

In [2]:
!pip install tfds-nightly

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds


ds = tfds.load('lost_and_found', split='train', batch_size=200)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

# 데이터 로드 

In [ ]:
dataset = next(iter(ds))

In [ ]:
print(dataset.keys())

In [ ]:
images = dataset['image_left'].numpy()
labels = dataset['segmentation_label'].numpy()

In [ ]:
print(images.shape)
print(labels.shape)

In [ ]:
print(np.max(labels), np.min(labels))

In [ ]:
#로드 한 데이터 확인 
import matplotlib.pyplot as plt
plt.imshow(images[0])
plt.show()

plt.imshow(labels[0].squeeze())
plt.show()

# 전처리 

## label 전처리

- 배경이 아닌 것(>0)은 전부 전경(1)으로 처리한다.
- label을 명확히 하기 위함 

In [ ]:
labels[labels>0]=1

In [ ]:
plt.imshow(labels[0].squeeze(),'gray')

## 리사이즈

In [ ]:
print(images.shape, labels.shape)

In [ ]:
import cv2 
raw_x = []
for i in range(len(images)):
  temp = cv2.resize(images[i],dsize = (256,256))
  raw_x.append(temp)
raw_x = np.array(raw_x)

raw_y = []
for i in range(len(images)):
  temp = cv2.resize(labels[i],dsize = (256,256))
  raw_y.append(temp)
raw_y = np.array(raw_y)

In [ ]:
raw_y = raw_y.reshape(200,256,256,1)

In [ ]:
print(raw_x.shape, raw_y.shape)

## 정규화 

In [ ]:
x = (raw_x/255).astype(np.float)
y = (raw_y/255).astype(np.int)

## 데이터 분할 

In [ ]:
from sklearn.model_selection import train_test_split
train_x,test_x, train_y,test_y = train_test_split(x,y, random_state=42)

# 모델링

## 모듈 임포트 

In [ ]:
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, MaxPooling2D, Conv2DTranspose, concatenate
from tensorflow.keras import backend as keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler

def unet(input_size=(256,256,3)): #train_x, test_x 는 256,256,3의 shape을 가져야 함 
    inputs = Input(input_size)
    
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    conv1 = BatchNormalization()(conv1)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
    conv1 = BatchNormalization()(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
    conv2 = BatchNormalization()(conv2)
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
    conv2 = BatchNormalization()(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
    conv3 = BatchNormalization()(conv3)
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)
    conv3 = BatchNormalization()(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool3)
    conv4 = BatchNormalization()(conv4)
    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv4)
    conv4 = BatchNormalization()(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(pool4)
    conv5 = BatchNormalization()(conv5)
    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv5)
    conv5 = BatchNormalization()(conv5)
    
    up = Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv5)
    up6 = concatenate([up, conv4], axis=3) # <---------- Conv4 를 넣음 
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(up6)
    conv6 = BatchNormalization()(conv6)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv6)
    conv6 = BatchNormalization()(conv6)
    
    up = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv6)
    up7 = concatenate([up, conv3], axis=3) # <------------ Conv 3를 넣음 
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(up7)
    conv7 = BatchNormalization()(conv7)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv7)
    conv7 = BatchNormalization()(conv7)
    
    up = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv7)
    up8 = concatenate([up, conv2], axis=3)
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(up8)
    conv8 = BatchNormalization()(conv8)
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv8)
    conv8 = BatchNormalization()(conv8)
    
    up = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(conv8) #<--- maxpooling과 반대의 역할을 함, 앞선 conv1과 반대의 작용을 하기 때문 
    up9 = concatenate([up, conv1], axis=3)
    conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(up9)
    conv9 = BatchNormalization()(conv9)
    conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv9)
    conv9 = BatchNormalization()(conv9)

    conv10 = Conv2D(1, (1, 1), activation='sigmoid')(conv9) #<-------- 1장으로 만듬 

    return Model(inputs=[inputs], outputs=[conv10])

## 모델 컴파일, 학습

In [ ]:
model = unet()

model.compile(optimizer = 'adam', loss = 'mse', metrics = ['accuracy'])

model.fit(train_x,train_y, batch_size = 32, epochs = 20, verbose=1, validation_split = 0.2)

## 모델 검증 

In [ ]:
fig, loss_ax = plt.subplots()

acc_ax = loss_ax.twinx()

loss_ax.plot(hist.history['loss'], 'y', label='train loss')
loss_ax.plot(hist.history['val_loss'], 'r', label='val loss')
loss_ax.set_ylim([0.0, 1.0])

acc_ax.plot(hist.history['accuracy'], 'b', label='train acc')
acc_ax.plot(hist.history['val_accuracy'], 'g', label='val acc')
acc_ax.set_ylim([0.0, 1.0])

loss_ax.set_xlabel('epoch')
loss_ax.set_ylabel('loss')
acc_ax.set_ylabel('accuray')

loss_ax.legend(loc='upper left')
acc_ax.legend(loc='lower left')

plt.show()

## 예측 

In [ ]:
for i in range(5):
  plt.subplot(1,2,1)
  plt.imshow(test_y[i].squeeze(),'gray')
  plt.subplot(1,2,2)
  plt.imshow(y_[i].squeeze(),'gray')
  plt.show()